<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Ensenble-System" data-toc-modified-id="Ensenble-System-1">Ensenble System</a></span></li><li><span><a href="#データ読み取り" data-toc-modified-id="データ読み取り-2">データ読み取り</a></span></li><li><span><a href="#Bagging" data-toc-modified-id="Bagging-3">Bagging</a></span></li><li><span><a href="#Boosting" data-toc-modified-id="Boosting-4">Boosting</a></span></li><li><span><a href="#Stacking" data-toc-modified-id="Stacking-5">Stacking</a></span></li></ul></div>

- Titanic
- https://www.kaggle.com/c/titanic

In [44]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Ensenble System

# データ読み取り

In [4]:
print(pd.options.display.max_rows, 'default:60')
print(pd.options.display.max_columns, 'default:20')
pd.options.display.max_columns = 200
print(pd.options.display.max_rows)
print(pd.options.display.max_columns)

60 default:60
200 default:20
60
200


In [10]:
path_to_train_csv = '../data/train.csv'
df = pd.read_csv( path_to_train_csv,
                  index_col='PassengerId')
print(df.shape)
print("乗客員：", len(df.index))
print("特徴量：", len(df.columns))
df.head(2)

(891, 11)
乗客員： 891
特徴量： 11


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [12]:
df_2 = df.drop(labels=['Age', 'Cabin'],
        axis=1)
df_3 = df_2.dropna(axis=0,
            subset=['Embarked'])
df_4 = df_3.drop(labels='Name',
                 axis=1)
# カテゴリデータの処理
df_5 = pd.get_dummies(df_4[df_4.columns[1:]])
df_6 = df_4.drop(labels='Ticket',
                 axis=1)
df_7 = pd.get_dummies(df_6[df_6.columns[1:]])

In [17]:
X = df_7.values
y = df_6[df_6.columns[0]].values
print(X.shape, y.shape)

(889, 9) (889,)


データ分割

In [18]:
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=0)

標準化

In [21]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

# Bagging

In [24]:
seed = 7
# k-fold cross-validation
kfold = KFold( n_splits=5,
               random_state=seed)

In [26]:
# Define a decision tree classifier
cart = DecisionTreeClassifier()
num_trees = 100

In [68]:
# create classification model for bagging
model = BaggingClassifier( base_estimator=cart,
                           n_estimators=num_trees,
                           random_state=seed)

In [32]:
results = cross_val_score(model, X_train_std, y_train, cv=kfold)

In [33]:
for i in range(len(results)):
    print('Model: ', i, ' Accuracy is: ', results[i])

Model:  0  Accuracy is:  0.784
Model:  1  Accuracy is:  0.8
Model:  2  Accuracy is:  0.822580645161
Model:  3  Accuracy is:  0.766129032258
Model:  4  Accuracy is:  0.83064516129


In [40]:
from statistics import mean
mean(results)

0.80067096774193547

# Boosting

In [38]:
# define a decision tree classifier
cart = DecisionTreeClassifier()
num_trees = 25

# create classification model for boosting
model = AdaBoostClassifier( base_estimator=cart,
                            n_estimators=num_trees,
                            learning_rate=0.1)
# fit
model.fit(X_train_std, y_train)

pred_label = model.predict(X_test_std)
nnz = np.float(np.shape(y_test)[0] - np.count_nonzero(pred_label - y_test))
acc = 100*nnz/np.shape(y_test)[0]

print('accuracy is: ', acc)

accuracy is:  74.53183520599251


# Stacking

In [41]:
# define a method to caluculate accuracy
def CalculateAccuracy(y_test, pred_label):
    nnz = np.shape(y_test)[0] - np.count_nonzero(pred_label - y_test)
    acc = 100*nnz/float(np.shape(y_test)[0])
    return acc

In [47]:
# clf:classifier
clf1 = KNeighborsClassifier(n_neighbors=2)
clf2 = RandomForestClassifier(n_estimators=2,
                              random_state=1)
clf3 = GaussianNB()

In [56]:
lr = LogisticRegression()

In [48]:
clf1.fit(X_train_std, y_train)
clf2.fit(X_train_std, y_train)
clf3.fit(X_train_std, y_train)

GaussianNB(priors=None)

train accuracy

In [60]:
f1 = clf1.predict(X_train_std)
acc1 = CalculateAccuracy(y_train, f1)
print('accuracy from KNN: ', acc1)

f2 = clf2.predict(X_train_std)
acc2 = CalculateAccuracy(y_train, f2)
print('accuracy from KNN: ', acc2)

f3 = clf3.predict(X_train_std)
acc3 = CalculateAccuracy(y_train, f3)
print('accuracy from KNN: ', acc3)

accuracy from KNN:  87.13826366559485
accuracy from KNN:  89.06752411575563
accuracy from KNN:  80.2250803858521


In [61]:
f = [f1, f2, f3]
f = np.transpose(f)

lr.fit(f, y_train)
final = lr.predict(f)
acc4 = CalculateAccuracy(y_train, final)
print('accuracy from stacking: ', acc4)

accuracy from stacking:  89.87138263665595


test accuracy

In [65]:
f1 = clf1.predict(X_test_std)
acc1 = CalculateAccuracy(y_test, f1)
print('accuracy from KNN: ', acc1)

f2 = clf2.predict(X_test_std)
acc2 = CalculateAccuracy(y_test, f2)
print('accuracy from KNN: ', acc2)

f3 = clf3.predict(X_test_std)
acc3 = CalculateAccuracy(y_test, f3)
print('accuracy from KNN: ', acc3)

accuracy from KNN:  74.53183520599251
accuracy from KNN:  77.90262172284645
accuracy from KNN:  76.02996254681648


In [67]:
f = [f1, f2, f3]
f = np.transpose(f)

lr.fit(f, y_test)
final = lr.predict(f)
acc4 = CalculateAccuracy(y_test, final)
print('accuracy from stacking: ', acc4)

accuracy from stacking:  76.02996254681648
